### Find albums in Spotify and add songs to a playlist

In [1]:
import requests
import json
!pip install pandas
import pandas as pd
!pip install jsontable
import jsontable
import io
import time

     |████████████████████████████████| 9.9 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 68.3 MB/s eta 0:00:01


### Set Global Date Variable
Allows specification of which source CSV to pull and helps to ensure unique playlists are created

In [2]:
pullDate = "20210306"

### Token Management
Right now I have to manually refresh the token via https://developer.spotify.com/console

In [3]:
TOKEN = "BQAc7RZZ4_PVL1765W0kurFCxC0h6wy0aXUMvlcEw9HfTsQcdWiRpqvk19Q-SP3Yylr4nN8gjkovJVzvwZVk6IVAZWuf-jv_nm5ScG1mG2aKr4tCAFo69K_Xs5TATAY07QRxlLL0AUJH4SgWsL3y2t_QIyFShlINHGs7vdriTboM7fyZEgSOlil5Vb3mEbhk-efvmME9Ow-I1n1dmdrDbVmUUAuTPDqly-g"

### Header Construction

In [4]:
header = {"Authorization": "Bearer {}".format(TOKEN)}

### Request Construction
> Remember that Spotify will return a max of 20 objects by default, so artists with lots of albums might not truly be returning the most recent

In [5]:
artist = "bastille"
artist = artist.replace(" ", "%20")
artist = "artist:%22" + artist + "%22"
# album = "Bad Blood"
# album = album.replace(" ", "%20")
# album = "album:%22" + album + "%22"
# queryParam = album + artist
target = "https://api.spotify.com/v1/search?q={}&type=album".format(artist)
request = requests.get(target, headers = header).json()
print(json.dumps(request, indent=2))

{
  "albums": {
    "href": "https://api.spotify.com/v1/search?query=artist%3A%22bastille%22&type=album&offset=0&limit=20",
    "items": [
      {
        "album_type": "album",
        "artists": [
          {
            "external_urls": {
              "spotify": "https://open.spotify.com/artist/7EQ0qTo7fWT7DPxmxtSYEc"
            },
            "href": "https://api.spotify.com/v1/artists/7EQ0qTo7fWT7DPxmxtSYEc",
            "id": "7EQ0qTo7fWT7DPxmxtSYEc",
            "name": "Bastille",
            "type": "artist",
            "uri": "spotify:artist:7EQ0qTo7fWT7DPxmxtSYEc"
          }
        ],
        "available_markets": [
          "US"
        ],
        "external_urls": {
          "spotify": "https://open.spotify.com/album/64fQ94AVziavTPdnkCS6Nj"
        },
        "href": "https://api.spotify.com/v1/albums/64fQ94AVziavTPdnkCS6Nj",
        "id": "64fQ94AVziavTPdnkCS6Nj",
        "images": [
          {
            "height": 640,
            "url": "https://i.scdn.co/image/a

### Function Accepts CSV as Input and Pulls New Albums

Testing out artist searches that resulted in errors

In [11]:
newArtist = "L'FREAQ"
newArtist = newArtist.replace(" ", "%20")
newArtist = newArtist.replace("&", "%26")
newArtist = newArtist.replace("'", "%27")
newArtist = newArtist.replace(".", "%2E")
newArtist = "artist:%22" + newArtist + "%22"
target = "https://api.spotify.com/v1/search?q={}&type=album".format(newArtist)
print(target)
request = requests.get(target, headers = header).json()
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
print(len(df.index))
print(df)
# paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
# converter = jsontable.converter()
# converter.set_paths(paths)
# resultTable = converter.convert_json(request)
# df = pd.DataFrame(data=resultTable)
# df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
# df = df.drop(df.index[0])
# df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
# newest = pd.DataFrame(data=df.iloc[[0]])

https://api.spotify.com/v1/search?q=artist:%22L%27FREAQ%22&type=album
1
    0           1       2             3
0  ID  Album Name  Tracks  Release Date


In [6]:
url = "https://raw.githubusercontent.com/patrick-still/music-engine/main/New%20Music%20Source%20Files/iTunes%20Alternative%20{}.csv".format(pullDate)
# url = "https://raw.githubusercontent.com/patrick-still/music-engine/main/sampleArtists.csv"
fetch = requests.get(url).content
artists = pd.read_csv (io.StringIO(fetch.decode('utf-8')))
artists.style

newAlbums = pd.DataFrame()
errors = []

def pullAlbums(artistDF):
    
    global newAlbums
    global errors
    
    for index, row in artistDF.iterrows():
        # Wait five seconds to avoid 20 calls/minute limit on iTunes API
        # time.sleep(5)
        newArtist = row["Artist"]
        newArtist = newArtist.replace(" ", "%20")
        newArtist = newArtist.replace("&", "%26")
        newArtist = newArtist.replace("'", "%27")
        newArtist = newArtist.replace(".", "%2E")
        newArtist = "artist:%22" + newArtist + "%22"
        target = "https://api.spotify.com/v1/search?q={}&type=album".format(newArtist)
        request = requests.get(target, headers = header).json()
        paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
        converter = jsontable.converter()
        converter.set_paths(paths)
        resultTable = converter.convert_json(request)
        df = pd.DataFrame(data=resultTable)
        df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
        if len(df.index) > 1:
            df = df.drop(df.index[0])
            df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
            newest = pd.DataFrame(data=df.iloc[[0]])
            newAlbums = newAlbums.append(newest)
        else:
            errors.append(row["Artist"])
            continue

    print(newAlbums)
    # newAlbums.style
    # Would prefer to have the output styled in this manner but am getting "style is not supported for non-unique indices"
    print(errors)
    
pullAlbums(artists)

                  Album ID                                         Album Name  \
4   6Tcx0miDDNxBuD7p4ySabf          3,000 Miles From Home. (Acoustic Version)   
3   2NgnnfzxHXketoOg1iU9uw               alchemist rats beg bashful (remixes)   
3   1ECjZ9nOYJRl2csoAfam1j  All About You (feat. Foster The People) [The K...   
7   5TE0TunG17dIPzrsGZAtjp                                   As Days Get Dark   
6   52abbUrmRs1bmdVp01U9e2                                    BEAUTY IN DEATH   
..                     ...                                                ...   
3   74A6O0ISRJ7udEVOEl79yO                             Starting To Get To You   
3   0Ztyr1KSM2PmsHzcfYT4mC                                             Unholy   
6   6pEcL2I73C50n4NKrmO3cG                                         Essentials   
2   3L8Me5z5XbMMkFTtyqzCaj                                              Co-op   
3   3W5tSRICU2loEbYx0Cpv8u                                         Go Outside   

   Tracks Release Date  
4 

In [7]:
display(newAlbums)

,Album ID,Album Name,Tracks,Release Date
4,6Tcx0miDDNxBuD7p4ySabf,"3,000 Miles From Home. (Acoustic Version)",1,2021-03-05
3,2NgnnfzxHXketoOg1iU9uw,alchemist rats beg bashful (remixes),18,2021-03-05
3,1ECjZ9nOYJRl2csoAfam1j,All About You (feat. Foster The People) [The K...,1,2021-03-05
7,5TE0TunG17dIPzrsGZAtjp,As Days Get Dark,11,2021-03-05
6,52abbUrmRs1bmdVp01U9e2,BEAUTY IN DEATH,12,2021-03-05
...,...,...,...,...
3,74A6O0ISRJ7udEVOEl79yO,Starting To Get To You,1,2021-03-02
3,0Ztyr1KSM2PmsHzcfYT4mC,Unholy,1,2021-03-02
6,6pEcL2I73C50n4NKrmO3cG,Essentials,15,2021-03-05
2,3L8Me5z5XbMMkFTtyqzCaj,Co-op,1,2021-03-01


### Pull Album ID Out of Response
Parse some key fields, and sort by both release date and number of tracks

In [5]:
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
df = df.drop(df.index[0])
df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
newest = pd.DataFrame(data=df.iloc[[0]])
newAlbums = pd.DataFrame()
newAlbums = newAlbums.append(newest)
newAlbums.style

,Album ID,Album Name,Tracks,Release Date
2,5G6oMu9zNW2acdV0lqzI3L,All This Bad Blood,25,2013-01-01


### Create Spotify Playlist
> Be careful here because Spotify will let you create playlists with the same name and description, assigning it a new playlist ID

In [8]:
userID = "t1e947vjw65dme71utp690f50"
createPlaylistTarget = "https://api.spotify.com/v1/users/{}/playlists".format(userID)
payload = {"name": pullDate, "description": "Created from Jupyter notebook", "public": False}
# payload = {"name": "Test Playlist", "description": "Created from Jupyter notebook", "public": False}
payload = json.dumps(payload, indent=2)
createPlaylistPOST = requests.post(createPlaylistTarget, headers = header, data = payload).json()
print(json.dumps(createPlaylistPOST, indent=2))

{
  "collaborative": false,
  "description": "Created from Jupyter notebook",
  "external_urls": {
    "spotify": "https://open.spotify.com/playlist/1jfpTJTNYLJkmXNqUXoHKH"
  },
  "followers": {
    "href": null,
    "total": 0
  },
  "href": "https://api.spotify.com/v1/playlists/1jfpTJTNYLJkmXNqUXoHKH",
  "id": "1jfpTJTNYLJkmXNqUXoHKH",
  "images": [],
  "name": "20210306",
  "owner": {
    "display_name": "Patrick",
    "external_urls": {
      "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
    },
    "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
    "id": "t1e947vjw65dme71utp690f50",
    "type": "user",
    "uri": "spotify:user:t1e947vjw65dme71utp690f50"
  },
  "primary_color": null,
  "public": false,
  "snapshot_id": "MSxiZjJlN2ZjNmIwNWRkNGE0MTJjNmI4MjU3MzcwYmE5YWQ1ZjIxYjU3",
  "tracks": {
    "href": "https://api.spotify.com/v1/playlists/1jfpTJTNYLJkmXNqUXoHKH/tracks",
    "items": [],
    "limit": 100,
    "next": null,
    "offset":

Retrieve User Information

In [19]:
getUserTarget = "https://api.spotify.com/v1/users/{}".format(userID)
getUser = requests.get(getUserTarget, headers = header).json()
print(json.dumps(getUser, indent=2))

{
  "display_name": "Patrick",
  "external_urls": {
    "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
  },
  "followers": {
    "href": null,
    "total": 1
  },
  "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
  "id": "t1e947vjw65dme71utp690f50",
  "images": [
    {
      "height": null,
      "url": "https://i.scdn.co/image/ab6775700000ee85804e93a20b51ae91923b1a51",
      "width": null
    }
  ],
  "type": "user",
  "uri": "spotify:user:t1e947vjw65dme71utp690f50"
}


### Fetch Track IDs for Album
Store tracks in a dataframe and potentially sort on popularity (not seeing clear metadata to support this) 
Probably makes sense to add to the playlist in this loop so that it's adding chunks of tracks per album and any kind of error handling can be associated with the album.

In [9]:
playlistID = "1jfpTJTNYLJkmXNqUXoHKH"
newTracks = pd.DataFrame()
popVals = []

def assembleTracks(albumDF, playlist):
    
    global newTracks
    global popVals
    
    # Assemble request to add the album's tracks to the target playlist which is passed as function parameter
    addTracksTarget = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlist)
    
    # For each album object, fetch its tracks and add their IDs and names to global dataframe
    for index, row in albumDF.iterrows():
        # Wait one second to avoid rate limit
        time.sleep(1)
        print("Working on " + row["Album Name"] + " (" + row["Album ID"] + ")")
        newAlbumID = row["Album ID"]
        fetchTrackTarget = "https://api.spotify.com/v1/albums/{}/tracks".format(newAlbumID)
        fetchTrack = requests.get(fetchTrackTarget, headers = header)
        callStatus = fetchTrack.status_code
        if callStatus != 200:
            print(callStatus)
            print(fetchTrack.text)
            # print(fetchTrack.headers)
            print("Problem fetching tracks for " + row["Album Name"] + " (" + row["Album ID"] + ")")
        fetchTrack = fetchTrack.json()
        # parse JSON for appropriate track metadata to store
        paths = [{"$.items.id":"Track ID"}, {"$.items.name":"Track Name"}]
        converter = jsontable.converter()
        converter.set_paths(paths)
        trackTable = converter.convert_json(fetchTrack)
        df = pd.DataFrame(data=trackTable)
        df.columns = ["Track ID", "Track Name"]
        df = df.drop(df.index[0])
        newTracks = newTracks.append(df)

        # Assemble track URIs to add to payload for request to add tracks to playlist
        trackURIs = []
        
        # For each track in the album add URI to list, then add set of tracks to Spotify playlist
        for index, row in df.iterrows():
            selectedTrack = row["Track ID"]
            selectedTrackURI = "spotify:track:{}".format(selectedTrack)
            trackURIs.append(selectedTrackURI)
            
            # Retrieve popularity for each track
            
            getPopularityTarget = "https://api.spotify.com/v1/tracks/{}".format(selectedTrack)
            trackDetails = requests.get(getPopularityTarget, headers = header).json()
            
            # Parse popularity values and add to global list that will be appended to newTracks dataframe
            
            paths = [{"$.popularity":"Popularity"}]
            converter = jsontable.converter()
            converter.set_paths(paths)
            popularityColumn = converter.convert_json(trackDetails)
            tempPop = pd.DataFrame(data=popularityColumn)
            tempPop = tempPop.drop(tempPop.index[0])
            newVal = tempPop.iloc[0][0]
            popVals.append(newVal)
        
        # Assemble payload to add album's worth of tracks to Spotify playlist
        payload = {"uris": trackURIs, "position":"null"}
        payload = json.dumps(payload, indent=2)
        
        # Execute request to update playlist
        addTracks = requests.post(addTracksTarget, headers = header, data = payload).json()
    
    # Full logic has run, adding accumulated popularity values as a column in newTracks dataframe
    newTracks["Popularity"] = popVals    
    display(newTracks)
        
assembleTracks(newAlbums, playlistID)

Working on 3,000 Miles From Home. (Acoustic Version) (6Tcx0miDDNxBuD7p4ySabf)
Working on alchemist rats beg bashful (remixes) (2NgnnfzxHXketoOg1iU9uw)
Working on All About You (feat. Foster The People) [The Knocks VIP] (1ECjZ9nOYJRl2csoAfam1j)
Working on As Days Get Dark (5TE0TunG17dIPzrsGZAtjp)
Working on BEAUTY IN DEATH (52abbUrmRs1bmdVp01U9e2)
Working on BORED (0kb2pt4dfB3si7zl2mYjo5)
Working on Break My Baby - Stripped Back (2NI3n21RjO0N66BhNg6G5p)
Working on Falling Up (2KUon4joIVloHPwuL4peKU)
Working on Cedars (0NE0GWEyzHUZChCw6HvIVl)
Working on Close To You (0cow9HqVjJlNgDYUBkY6m2)
Working on Confident (63mnfDbOjkcAPVewgAEFSB)
Working on Cookies & Cream (6Yxikm8Vp8EumDkRwbEp2L)
Working on VGM Covers, Vol. 3 (1bWXyunugbM8rtzZVf0cNU)
Working on Cosy Karaoke, Vol. 1 (4a0t98Zz2S4p3CSe4jf4Sv)
Working on Dirty Gonzo (0bbx1gJ1vU9uChdYXAjTmi)
Working on Dizzy Demos: 2 Tickets to Cheeseburger in Paradise (5sVqmXQFGP5P9EjLkKaJM2)
Working on Emergency Telephone (5lbnFvJgYtqN9UPCVPQu2U)
Wor

,Track ID,Track Name,Popularity
1,5LCJicrKffS6v5VHoddb9Y,"3,000 Miles From Home. - Acoustic Version",0
1,5HD0rRdfrJqFLkc0Ndqywi,jhariah dies and goes to hell,0
2,0Rk7sLHL0fanjEV4HfnI3i,(rat castle) (fantasyluv dreamix),0
3,5nTaPMlorNczA3QL3K7Wrs,(FOREVER!!!!!!!!!) [WOW OK Remix],0
4,6Q4scPCM8WzyOdHj9hPr1L,Blood Rivers Remix,0
...,...,...,...
13,2XD37WMljWtjWgPKvxK9GM,Og,0
14,0pNEdZxmeeC29QdPZg0V6b,Suck My Nutz,0
15,3a5OFQTF3DA1El2h9IHwUW,Fuck Shit,0
1,5wrYMoKrMCFlvPah7UUFHh,Co-op,18


In [10]:
pd.set_option("display.max_rows", None)
display(newTracks)

,Track ID,Track Name,Popularity
1,5LCJicrKffS6v5VHoddb9Y,"3,000 Miles From Home. - Acoustic Version",0
1,5HD0rRdfrJqFLkc0Ndqywi,jhariah dies and goes to hell,0
2,0Rk7sLHL0fanjEV4HfnI3i,(rat castle) (fantasyluv dreamix),0
3,5nTaPMlorNczA3QL3K7Wrs,(FOREVER!!!!!!!!!) [WOW OK Remix],0
4,6Q4scPCM8WzyOdHj9hPr1L,Blood Rivers Remix,0
5,762gG4XdlvvkTA7jR2YL6W,Dallas (jackson from online cowboy billionare ...,0
6,3niiFf9WUYPhFpPdWGPl15,neon glow (floral tattoo version),0
7,6QNMc4IiykgeWl2bFchlGU,yoshis island (world 7x7 mix),0
8,1H2BT2GKElGUb4BIIvpjNA,Calico (Emo Rocksteady Cover),0
9,0SDsOyOnndEw8ldxFl5vlk,Planetarium,0


### Add Tracks to a Playlist
POST https://api.spotify.com/v1/playlists/{playlist_id}/tracks  
Need the playlist ID and a list of track URIs  

> Folded this into the assembleTracks function, so the logic below is now just a reference for making a single call

In [9]:
playlistID = "25NbFuQM1kDleGz1bjGJ2x"
addTrackTarget = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlistID)
payload = {"uris": ["spotify:track:57As4lhkpp6tB422daefKK","spotify:track:3ePeWP6f8ccQtF7wayoNrD"], "position":"null"}
payload = json.dumps(payload, indent=2)
addTrack = requests.post(addTrackTarget, headers = header, data = payload).json()
print(json.dumps(addTrack, indent=2))

{
  "snapshot_id": "MTIsMzBmODE0NzUwOGVlNGJmZWZmZWYyMjExOGI2NzYyNjQzNzY5ZDExNA=="
}
